# About this notebook
This notebook is used for testing code related to allocating the EIA-923 data before integrating it into PUDL and the data pipeline. 

I've also been using this to simply load EIA tables from the PUDL database to explore the EIA data as it exists in PUDL.

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append('../../hourly-egrid/')

# Useful high-level external modules.
import numpy as np
import pandas as pd
import sqlalchemy as sa

import pudl

import src.data_cleaning as data_cleaning


In [ ]:
year = 2020

pudl_db = 'sqlite:///../data/downloads/pudl/pudl_data/sqlite/pudl.sqlite'
pudl_engine = sa.create_engine(pudl_db)

pudl_out = pudl.output.pudltabl.PudlTabl(
    pudl_engine,
    freq='MS',
    start_date=f'{year}-01-01',
    end_date=f'{year}-12-31'
)

## Load EIA data

In [ ]:
# extract all of the tables from pudl_out early in the process and select
# only the columns we need. this is for speed and clarity.

remove_this = pudl.analysis.allocate_net_gen
drop_interim_cols = False

IDX_GENS = ["report_date", "plant_id_eia", "generator_id"]
"""Id columns for generators."""

IDX_PM_ESC = ["report_date", "plant_id_eia", "energy_source_code", "prime_mover_code"]
"""Id columns for plant, prime mover & fuel type records."""

IDX_ESC = ["report_date", "plant_id_eia", "energy_source_code"]

IDX_U_ESC = ["report_date", "plant_id_eia", "energy_source_code", "unit_id_pudl"]

# extract all of the tables from pudl_out early in the process and select
# only the columns we need. this is for speed and clarity.
# extract all of the tables from pudl_out early in the process and select
# only the columns we need. this is for speed and clarity.
gf = (
    pudl_out.gf_eia923()
    .loc[
        :,
        IDX_PM_ESC
        + [
            "net_generation_mwh",
            "fuel_consumed_mmbtu",
            "fuel_consumed_for_electricity_mmbtu",
        ],
    ]
    .pipe(remove_this.manually_fix_energy_source_codes)
)
bf = (
    pudl_out.bf_eia923()
    .merge(
        pd.read_sql("boilers_entity_eia", pudl_out.pudl_engine),
        how="left",
        on=["plant_id_eia", "boiler_id"],
    )
    .loc[:, IDX_PM_ESC + ["fuel_consumed_mmbtu"]]
)
gen = (
    pudl_out.gen_original_eia923().loc[:, IDX_GENS + ["net_generation_mwh"]]
    # removes 4 records with NaN generator_id as of pudl v0.5
    .dropna(subset=IDX_GENS)
)
gens = pudl_out.gens_eia860().loc[
    :,
    IDX_GENS
    + [
        "prime_mover_code",
        "unit_id_pudl",
        "capacity_mw",
        "fuel_type_count",
        "operational_status",
        "retirement_date",
    ]
    + list(pudl_out.gens_eia860().filter(like="energy_source_code"))
    + list(pudl_out.gens_eia860().filter(like="startup_source_code")),
]
# add any startup energy source codes to the list of energy source codes
gens = remove_this.add_startup_energy_sources_to_esc_columns(gens)


In [ ]:
plant_id_to_investigate = 160

In [ ]:
gens[gens['plant_id_eia'] == plant_id_to_investigate]

In [ ]:
gens.loc[gens['prime_mover_code'].isna(), ['plant_id_eia','generator_id']].drop_duplicates()

In [ ]:
gens[gens['generator_id'].str.contains('ST')]

In [ ]:
gen[gen['plant_id_eia'] == plant_id_to_investigate]

In [ ]:
gf[gf['plant_id_eia'] == plant_id_to_investigate]